In [1]:
import rebound
import sys
#change the next line to reflect where you have downloaded the source code
sys.path.insert(0, '/Users/kvolk/Documents/GitHub/SBDynT/src')
import sbdynt as sbd

**Example of setting up an integration with just the planets at a spefied epoch**

Note that if not all planets are included in a simulation, we correct for that, so this routine also returns the necessary cartesian correction to the heliocentric position and velocity vectors of any small bodies you want to add to the simulation. 

You must use these corrections *before* using a sim.move_to_com() command!




In [2]:
sim= rebound.Simulation()
epoch = 2457217.5
flag, sim, dx, dy, dz, dvx, dvy, dvz = sbd.add_planets(sim,epoch=epoch)
if(flag):
    print("Only the giant planets are included, so the corrections are non-zero: ")
    print(dx,dy,dz)
    print(dvx,dvy,dvz)

Only the giant planets are included, so the corrections are non-zero: 
0.0 0.0 0.0
0 0.0 0.0


In [3]:
sim= rebound.Simulation()
flag, sim, dx, dy, dz, dvx, dvy, dvz = sbd.add_planets(sim,planets=['Jupiter','Saturn','Uranus', 'Neptune'],
                                                       epoch=epoch)
if(flag):
    print("Only the giant planets are included, so the corrections are non-zero: ")
    print(dx,dy,dz)
    print(dvx,dvy,dvz)
    
    

Only the giant planets are included, so the corrections are non-zero: 
-1.0867896335706758e-06 4.111305404909633e-06 1.1748741478494982e-08
-3.209377390988975e-05 -7.75337397002884e-06 7.561184199213491e-07


In [4]:
#when adding small bodies by observed heliocentric orbit, you should convert to cartesian using the GM
#value for the sun used for that heliocentric orbit fit and then add them to the simulation using 
#the above corrections

#Example, suppose you have an orbit calculated at the above epoch using Find_Orb with the following
#orbital elements:
epoch = 2457217.5
a = 39.4060972335126 #(semimajor axis in au)
ec = 0.19121388131996062 #(eccentricity)
inc=0.1279988198070029 #radians! (inclination relative to the ecliptic)
node = -2.3766713138417384 #radians! (longitude of ascending node)
argperi = 3.033977870435759 #radians! (argument of pericenter)
ma = 0.15683745626851575 #radians! (mean anomaly)

#You need the value of GMsun used by Find_Orb, which we have in a constants file
import hard_coded_constants as const

# We need to convert the Find_Orb orbit to heliocentric
# cartesian variables using Find_Orb's assumed solar GM
# which is in km^2/s^2, so have to convert a to km first
a = a/const.kmtoau
#now we use a built-in tool to convert to heliocentric cartesian
aeiflag, x, y, z, vx, vy, vz = sbd.aei_to_xv(GM=const.find_orb_sunGM, 
                a=a, e=ec, inc=inc, node=node, argperi=argperi, ma=ma)
# those positions and velocities are in km and km/s, so need to convert
# to au/year
x=x*const.kmtoau
y=y*const.kmtoau
z=z*const.kmtoau
vx=vx*const.kmtoau/const.stoyear
vy=vy*const.kmtoau/const.stoyear
vz=vz*const.kmtoau/const.stoyear

#now we can apply the corrections from add_planets to make them heliocentric
x+=dx; y+=dy; z+=dz;
vx+=dvx; vy+=dvy; vz+=dvz;

#finally we can add a test particle to sim with that corrected orbit:
sim.add(m=0., x=x, y=y, z=z, vx=vx, vy=vy, vz=vz, hash="test-orbit")

#now we can move to com
sim.move_to_com()

helio_orbit = sim.particles["test-orbit"].orbit(sim.particles[0])

print("Added heliocentric orbit: ")
print("a=",helio_orbit.a)
print("e=",helio_orbit.e)
print("i=",helio_orbit.inc)
print("node=",helio_orbit.Omega)
print("aperi=",helio_orbit.omega)
print("mean anomaly=",helio_orbit.M)


sim.status(showAllFields=False)


Added heliocentric orbit: 
a= 39.407558424179285
e= 0.19123915322497717
i= 0.12799646211122345
node= -2.376673552898542
aperi= 3.034146810180969
mean anomaly= 0.15671648069789423
---------------------------------
REBOUND version:     	4.3.2
REBOUND built on:    	Feb 19 2024 21:28:11
Number of particles: 	6
Selected integrator: 	ias15
Simulation time:     	0.0000000000000000e+00
Current timestep:    	0.400000
---------------------------------
<rebound.particle.Particle object at 0x7f94bea20840, m=1.0000059769986274 x=0.00346106016231451 y=0.0002763191852440362 z=-0.00015320621456877916 vx=0.0008681058230389302 vy=0.0023024789190979247 vz=-2.337071977160678e-05>
<rebound.particle.Particle object at 0x7f94bea207c0, m=0.0009547919099366768 x=-4.640079009063582 y=2.709933685630652 z=0.09249943531696177 vx=-1.4224164878812873 vy=-2.2502849100954925 vz=0.04117886591796361>
<rebound.particle.Particle object at 0x7f94bea20840, m=0.0002858856700231729 x=-4.527081593730463 y=-8.893180388894034 z=

In [5]:
# We have made a routine to do all the above for you for Find_Orb orbits! 
# Similar routines for other orbit fitting codes will be made at a future date
sim= rebound.Simulation()
obj_name='example'
#Find_orb orbital elements:
epoch = 2457217.5
a = 39.4060972335126 #(semimajor axis in au)
ec = 0.19121388131996062 #(eccentricity)
inc=0.1279988198070029 #radians! (inclination relative to the ecliptic)
node = -2.3766713138417384 #radians! (longitude of ascending node)
argperi = 3.033977870435759 #radians! (argument of pericenter)
ma = 0.15683745626851575 #radians! (mean anomaly)

flag, sim = sbd.initialize_from_heliocentric_Find_Orb_orbit(sim,des=obj_name,
                                                           a=a,e=ec,inc=inc,node=node,
                                                           aperi=argperi,ma=ma,
                                                           planets = ['jupiter','saturn','uranus','neptune'],
                                                           epoch=epoch)

helio_orbit = sim.particles[obj_name].orbit(sim.particles[0])

print("Added heliocentric orbit: ")
print("a=",helio_orbit.a)
print("e=",helio_orbit.e)
print("i=",helio_orbit.inc)
print("node=",helio_orbit.Omega)
print("aperi=",helio_orbit.omega)
print("mean anomaly=",helio_orbit.M)

Added heliocentric orbit: 
a= 39.407558424179285
e= 0.19123915322497717
i= 0.12799646211122345
node= -2.376673552898542
aperi= 3.034146810180969
mean anomaly= 0.15671648069789423
